# graphdatascience example

This notebook exemplifies how to use the `graphdatascience` Python library to operate Neo4j GDS.
It runs the FastRP+kNN end-to-end example from the GDS Manual, found [here](https://neo4j.com/docs/graph-data-science/current/end-to-end-examples/fastrp-knn-example).


## Prerequisites

Running this notebook requires a localhost Neo4j server with a recent GDS version (2.0+) installed.
We recommend using Neo4j Desktop.

Also required are the Python libraries neo4j ([Neo4j Python Driver](https://neo4j.com/docs/api/python-driver/current/index.html)) and of course the `graphdatascience` (see README for installation instructions).

In [ ]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
gds = GraphDataScience("bolt://localhost:7687", auth=None)

In [ ]:
# Create some data in Neo4j
gds.run_cypher(
    """
        CREATE
         (dan:Person {name: 'Dan'}),
         (annie:Person {name: 'Annie'}),
         (matt:Person {name: 'Matt'}),
         (jeff:Person {name: 'Jeff'}),
         (brie:Person {name: 'Brie'}),
         (elsa:Person {name: 'Elsa'}),

         (cookies:Product {name: 'Cookies'}),
         (tomatoes:Product {name: 'Tomatoes'}),
         (cucumber:Product {name: 'Cucumber'}),
         (celery:Product {name: 'Celery'}),
         (kale:Product {name: 'Kale'}),
         (milk:Product {name: 'Milk'}),
         (chocolate:Product {name: 'Chocolate'}),

         (dan)-[:BUYS {amount: 1.2}]->(cookies),
         (dan)-[:BUYS {amount: 3.2}]->(milk),
         (dan)-[:BUYS {amount: 2.2}]->(chocolate),

         (annie)-[:BUYS {amount: 1.2}]->(cucumber),
         (annie)-[:BUYS {amount: 3.2}]->(milk),
         (annie)-[:BUYS {amount: 3.2}]->(tomatoes),

         (matt)-[:BUYS {amount: 3}]->(tomatoes),
         (matt)-[:BUYS {amount: 2}]->(kale),
         (matt)-[:BUYS {amount: 1}]->(cucumber),

         (jeff)-[:BUYS {amount: 3}]->(cookies),
         (jeff)-[:BUYS {amount: 2}]->(milk),

         (brie)-[:BUYS {amount: 1}]->(tomatoes),
         (brie)-[:BUYS {amount: 2}]->(milk),
         (brie)-[:BUYS {amount: 2}]->(kale),
         (brie)-[:BUYS {amount: 3}]->(cucumber),
         (brie)-[:BUYS {amount: 0.3}]->(celery),

         (elsa)-[:BUYS {amount: 3}]->(chocolate),
         (elsa)-[:BUYS {amount: 3}]->(milk)
        """
)

In [ ]:
# Estimate memory of doing native graph loading
node_projection = ["Person", "Product"]
relationship_projection = {
    "BUYS": {"orientation": "UNDIRECTED", "properties": "amount"}
}
result = gds.graph.project.estimate(node_projection, relationship_projection)

print(f"Required memory for native loading: {result['requiredMemory']}")

In [ ]:
# Do native loading
G, _ = gds.graph.project("purchases", node_projection, relationship_projection)

print(f"Graph named '{G.name()}' projected")

In [ ]:
# Estimate memory of running FastRP
result = gds.fastRP.mutate.estimate(
    G,
    mutateProperty="embedding",
    embeddingDimension=4,
    relationshipWeightProperty="amount",
    iterationWeights=[0.8, 1, 1, 1],
)

print(f"Required memory for running FastRP: {result['requiredMemory']}")

In [ ]:
# Run FastRP and mutate projection with result
result = gds.fastRP.mutate(
    G,
    mutateProperty="embedding",
    embeddingDimension=4,
    relationshipWeightProperty="amount",
    iterationWeights=[0.8, 1, 1, 1],
)

print(f"Embedding vectors produced: {result['nodePropertiesWritten']}")

In [ ]:
# Run kNN and write back to db (skip mem. estimation this time...)
result = gds.knn.write(
    G,
    topK=2,
    nodeProperties=["embedding"],
    writeRelationshipType="SIMILAR",
    writeProperty="score",
)

print(f"Relationships produced: {result['relationshipsWritten']}")
print(f"Nodes compared: {result['nodesCompared']}")
print(f"Mean similarity: {result['similarityDistribution']['mean']}")

In [ ]:
# Rejoice in relationship in the Neo4j database!
query = """
        MATCH (p1:Person)-[r:SIMILAR]->(p2:Person)
        RETURN p1.name AS person1, p2.name AS person2, r.score AS similarity
        ORDER BY similarity DESCENDING, person1, person2
        """

# Print the most similar person nodes
gds.run_cypher(query)

In [ ]:
# Clean up database
gds.run_cypher("MATCH (n) DETACH DELETE n")
# Clean up graph catalog
G.drop()